PyQt5拥有丰富的控件。然而，没有哪个工具可以提供程序员在他们的应用程序中可能需要的所有部件。工具包通常只提供诸如按钮、文本组件、或滑块等最常见的控件。如果有需要一种更专业的控件，我们必须自己去创建它。

自定义控件是通过使用该工具包提供的绘图工具创建的。有两种基本的可行性：程序员可以修改或增强现有控件，或者他可以从头开始创建一个全新的自定义控件。

## Burning widget - 烧录控件

这是一个模拟烧录软件中的控件实例，我们可以在Nero，K3B或其他CD/ DVD刻录软件中参考看看。

In [1]:

import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QSlider, 
QHBoxLayout, QVBoxLayout)
from PyQt5.QtCore import Qt, QObject, pyqtSignal
from PyQt5.QtGui import QPainter, QFont, QColor, QPen

class Communicate(QObject):

    updateBW = pyqtSignal(int)

class BurningWidget(QWidget):

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):

        self.setMinimumSize(1, 30)
        self.value = 75
        self.num = [75, 150, 225, 300, 375, 450, 525, 600, 675]

    def setValue(self, value):

        self.value = value

    def paintEvent(self, e):

        qp = QPainter()
        qp.begin(self)
        self.drawWidget(qp)
        qp.end()

    def drawWidget(self, qp):

        font = QFont('Serif', 7, QFont.Light)
        qp.setFont(font)

        size = self.size()
        w = size.width()
        h = size.height()

        step = int(round(w / 10.0))

        till = int(((w / 750.0) * self.value))
        full = int(((w / 750.0) * 700))

        if self.value >= 700:

            qp.setPen(QColor(255, 255, 255))
            qp.setBrush(QColor(255, 255, 184))
            qp.drawRect(0, 0, full, h)
            qp.setPen(QColor(255, 175, 175))
            qp.setBrush(QColor(255, 175, 175))
            qp.drawRect(full, 0, till-full, h)

        else:

            qp.setPen(QColor(255, 255, 255))
            qp.setBrush(QColor(255, 255, 184))
            qp.drawRect(0, 0, till, h)

        pen = QPen(QColor(20, 20, 20), 1, Qt.SolidLine)

        qp.setPen(pen)
        qp.setBrush(Qt.NoBrush)
        qp.drawRect(0, 0, w-1, h-1)

        j = 0

        for i in range(step, 10*step, step):

            qp.drawLine(i, 0, i, 5)
            metrics = qp.fontMetrics()
            fw = metrics.width(str(self.num[j]))
            qp.drawText(i-fw/2, h/2, str(self.num[j]))
            j = j+1

class Example(QWidget):

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):

        sld = QSlider(Qt.Horizontal, self)
        sld.setFocusPolicy(Qt.NoFocus)
        sld.setRange(1, 750)
        sld.setValue(75)
        sld.setGeometry(30, 40, 150, 30)

        self.c = Communicate()
        self.wid = BurningWidget()
        self.c.updateBW[int].connect(self.wid.setValue)

        sld.valueChanged[int].connect(self.changeValue)
        hbox = QHBoxLayout()
        hbox.addWidget(self.wid)
        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addLayout(hbox)
        self.setLayout(vbox)

        self.setGeometry(300, 300, 390, 210)
        self.setWindowTitle('自定义控件')        
        self.show()

    def changeValue(self, value):

        self.c.updateBW.emit(value)
        self.wid.repaint()

if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = Example()
    app.exec_()

在这个例子中，我们创建了一个QSlider 和一个自定义控件，滑块控制自定义控件的变化。这个控件以图形方式显示给我们存储介质的总容量与完成容量。自定义控件的最小值为1，最大值为750。如果值达到700，我们开始用红色绘制。这通常表示光盘烧毁。

刻录控件被放置在窗口的底部，这是使用一个QHBoxLayout 和一个QVBoxLayout实现。
```
class BurningWidget(QWidget):

    def __init__(self):
        super().__init__()
```
刻录控件是基于QWidget 的控件。
```
        self.setMinimumSize(1, 30)
```
我们改变了控件的最小尺寸（高度）。默认值对我们来说有点小。
```
        font = QFont('Serif', 7, QFont.Light)
        qp.setFont(font)
```
我们用较小的字体作为默认值，这更适合我们的需求。
```
        size = self.size()
        w = size.width()
        h = size.height()

        step = int(round(w / 10.0))

        till = int(((w / 750.0) * self.value))
        full = int(((w / 750.0) * 700))
```
我们绘制的控件将会随窗口的变化而改变自身的大小，更大的窗口会得到更大的刻录控件，反之亦然。所以我们需获取窗口的大小，然后根据窗口的大小放置自定义控件。till 参数定义：绘制已烧录的大小，该值来自滑块控件，它是整个区域的比例。full 参数定义：在这个时候我们开始绘制红色。注意使用浮点算术实现绘制将会更加精确。

实际的绘制分三个步骤绘制黄色或红色和黄色矩形。然后，我们画垂直线，将控件分成几部分。最后，我们得出它指示介质的容量的数量。
```
            metrics = qp.fontMetrics()
            fw = metrics.width(str(self.num[j]))
            qp.drawText(i-fw/2, h/2, str(self.num[j]))
```
我们使用字体规格绘制文本。我们必须知道围绕垂直线的文本的宽度。
```
    def changeValue(self, value):

        self.c.updateBW.emit(value)
        self.wid.repaint()
```
当我们移动滑块，changeValue()方法被调用。里面的方法，我们发送自定义updateBW信号与参数，参数是滑块的当前值，该值用于计算烧录控件要绘制的容量。然后自定义控件重新绘制。

程序执行后 
这里写图片描述  这里写图片描述